To dos: 
    
    1) EDA on test see if it's similar with train
    
    2) FE add time compmenets according to different region e.g. day of week etc
    
    3) test impression list length

Notes:

    * total train shape: (15,932,992, 12)
    * no action type and reference is nan in train

In [1]:
import pandas as pd
import numpy as np
from utils import load_data

In [2]:
%%time
train = load_data('train', nrows=int(1e6))
# train = load_data('train', nrows=None)

CPU times: user 1.16 s, sys: 90.1 ms, total: 1.25 s
Wall time: 1.25 s


total train shape: (15,932,992, 12)

In [3]:
train.shape

(1000000, 12)

In [4]:
ref_act_size = train.groupby(['reference', 'action_type']).size()
dist = ref_act_size.groupby(level='action_type').value_counts()

In [5]:
# dist

In [6]:
# dist.iloc[dist.index.get_level_values('action_type') == 'clickout item']

In [7]:
# clickout_vc = train[train.action_type=='clickout item'].groupby('reference').size().value_counts()

In [57]:
ref_act = train[['session_id', 'reference', 'action_type']].copy()
ref_act = ref_act[~ref_act.reference.str.contains('[a-zA-Z]')].reset_index(drop=True)
unique_actions = ref_act['action_type'].unique()
action2natural = {v: k for k, v in enumerate(unique_actions)}
action_names = list(action2natural.keys())
actions = ref_act['action_type'].map(action2natural).values

actions_ohe = pd.DataFrame(np.eye(len(unique_actions), dtype=int)[actions], columns=action_names)

In [58]:
action2natural

{'interaction item image': 0,
 'clickout item': 1,
 'interaction item info': 2,
 'interaction item deals': 3,
 'interaction item rating': 4,
 'search for item': 5}

In [59]:
ref_act = pd.concat([ref_act, actions_ohe], axis=1)

In [60]:
ref_act.head()

,session_id,reference,action_type,interaction item image,clickout item,interaction item info,interaction item deals,interaction item rating,search for item
0,aff3928535f48,666856,interaction item image,1,0,0,0,0,0
1,aff3928535f48,666856,interaction item image,1,0,0,0,0,0
2,aff3928535f48,666856,interaction item image,1,0,0,0,0,0
3,aff3928535f48,109038,interaction item image,1,0,0,0,0,0
4,aff3928535f48,666856,interaction item image,1,0,0,0,0,0


In [61]:
ref_act[ref_act.session_id=='000263df674fa']

,session_id,reference,action_type,interaction item image,clickout item,interaction item info,interaction item deals,interaction item rating,search for item
772331,000263df674fa,8249,search for item,0,0,0,0,0,1
772332,000263df674fa,8249,clickout item,0,1,0,0,0,0
772333,000263df674fa,8249,clickout item,0,1,0,0,0,0


In [62]:
ref_act = ref_act.groupby(['session_id', 'reference', 'action_type']).sum().reset_index()
del ref_act['session_id']

In [71]:
M = [2]*6
s = []
for k, c in enumerate(action_names):
    print(c)
    # Compute the global mean
    mu = ref_act[c].mean()

    # Compute the number of values and the mean of each group
    agg = ref_act.groupby('reference')[c].agg(['count', 'mean'])
    counts = agg['count']
    mus = agg['mean']
    # Compute the "smoothed" means
    # smoothed = (n*mu + m*Mu)/(n+m)
    smoothed = (counts*mus + M[k]*mu)/(counts + M[k])
    s.append(smoothed.reset_index(name=c))
    print(s[-1].duplicated().sum())
#     ref_act[c] = ref_act['reference'].map(smoothed)
    #     # Replace each value by the according smoothed mean
#     return df[by].map(smooth)
dfs = [df.set_index('reference') for df in s]
encoding = pd.concat(dfs, axis=1).reset_index()


interaction item image
0
clickout item
0
interaction item info
0
interaction item deals
0
interaction item rating
0
search for item
0


In [70]:
encoding.head()

,reference,interaction item image,clickout item,interaction item info,interaction item deals,interaction item rating,search for item
0,1000029,5.682097,0.360390,0.064675,0.044002,0.049567,0.035450
1,10000338,2.682097,0.360390,0.064675,0.044002,0.049567,0.368783
2,1000041,1.005786,0.510146,0.024253,0.016501,0.268587,0.138294
3,1000065,2.841048,0.513528,0.365671,0.022001,0.024783,0.017725
4,10000706,3.761573,0.270293,0.298507,0.033002,0.037175,0.026588


clicked vs viewed

In [33]:
ref_imp = train.loc[train.action_type=='clickout item', ['reference', 'impressions']].reset_index(drop=True)
ref_imp = ref_imp[~ref_imp.reference.str.contains('[a-zA-Z]')].reset_index(drop=True)
# ref_imp = ref_imp[ref_imp]

In [34]:
ref_imp['impressions'] = ref_imp['impressions'].str.split('|')

In [35]:
%%time
ref_imp['impressions'] = ref_imp.apply(lambda row: list(set(row.impressions) - set([row.reference])), axis=1)

CPU times: user 3.03 s, sys: 12.2 ms, total: 3.04 s
Wall time: 3.04 s


In [36]:
ref_imp.head()

,reference,impressions
0,109038,"[109013, 1041528, 5100540, 6806806, 129343, 39..."
1,1257342,"[2050977, 1031578, 54824, 1332971, 129343, 390..."
2,2795374,"[107162, 1088390, 148884, 1258068, 3370484, 38..."
3,1032816,"[46363, 47410, 18425, 819616, 927627, 98737, 1..."
4,1032816,"[46363, 47410, 18425, 819616, 927627, 98737, 1..."


In [37]:
imps = ref_imp.impressions.values
imps = [j for i in imps for j in i]
click_imp = pd.concat([pd.Series([0]*len(imps), index=imps), 
                       pd.Series([1]*len(ref_imp), index=ref_imp.reference.values)])
click_imp.index.name = 'item_id'
click_imp = pd.DataFrame(click_imp, columns=['clicked']).reset_index()

In [73]:
click_imp.head()

,item_id,clicked
0,109013,0.065219
1,1041528,0.090739
2,5100540,0.104350
3,6806806,0.004350
4,129343,0.090584


In [39]:
mu = click_imp['clicked'].mean()
m = 2
agg = click_imp.groupby('item_id')['clicked'].agg(['count', 'mean'])
count = agg['count']
mus = agg['mean']
smoothed = (count*mus + m*mu)/(count + m)
click_imp['clicked'] = click_imp['item_id'].map(smoothed)

In [40]:
click_imp.head()

,item_id,clicked
0,109013,0.065219
1,1041528,0.090739
2,5100540,0.104350
3,6806806,0.004350
4,129343,0.090584


In [41]:
click_imp.duplicated().sum()

1954912

In [45]:
smoothed.reset_index(name='clicked').duplicated().sum()

0